In [8]:
import os, sys, time, random, urllib, subprocess, pickle, unidecode
import pandas as pd
from eyed3 import id3
from mutagen.id3 import ID3
from mutagen.easyid3 import EasyID3
from mutagen import File

### Build out music library objects

#### Initialize path and collect all artist directories

In [ ]:
path = "/Users/OGN/Music/iTunes/iTunes Music/Music/"
music_dict = dict()
music_lib = []

artist_dirs = [ os.path.join(path, i) for i in os.listdir(path) 
               if os.path.isdir(os.path.join(path, i))]

#### Create list of artist folders with their respective paths (as dicts)

In [ ]:
def create_music_lib(path):
    for at_dir in os.listdir(path):
        at_dir_path = os.path.join(path, at_dir)
        if os.path.isdir(at_dir_path):
            artist_dict = {
                "ARTIST_NAME": at_dir,
                "ARTIST_PATH": at_dir_path
            }   
            yield artist_dict
            
music_lib = list(create_music_lib(path))
music_lib

#### For each artist folder, list all album subfolders

In [ ]:
for artist in music_lib:
    album_dirs = [al for al in os.listdir(artist["ARTIST_PATH"])
                    if os.path.isdir(os.path.join(artist["ARTIST_PATH"], al))]
    
    artist['ALBUMS'] = []
    for album in album_dirs:
        album_dict = {
            "ALBUM_NAME": album,
            "ALBUM_PATH": os.path.join(artist['ARTIST_PATH'], album)
        }
        artist['ALBUMS'].append(album_dict)

In [ ]:
pickle.dump(music_lib, open("musicLib.p" , "wb"))

In [2]:
music_lib = pickle.load(open("musicLib.p", "rb"))

##### Data structure: music_lib[artist_index] = {artist: path, albums = albums[album_index]}

In [3]:
music_lib[1]['ALBUMS'][0]['ALBUM_PATH']
music_lib[:5]

[{'ALBUMS': [{'ALBUM_NAME': 'Lost Horizons',
    'ALBUM_PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Lemon Jelly/Lost Horizons'}],
  'ARTIST_NAME': 'Lemon Jelly',
  'ARTIST_PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Lemon Jelly'},
 {'ALBUMS': [{'ALBUM_NAME': 'WT 21',
    'ALBUM_PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Florian Kupfer/WT 21'},
   {'ALBUM_NAME': 'This Society',
    'ALBUM_PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Florian Kupfer/This Society'}],
  'ARTIST_NAME': 'Florian Kupfer',
  'ARTIST_PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Florian Kupfer'},
 {'ALBUMS': [{'ALBUM_NAME': 'From All Purity',
    'ALBUM_PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Indian/From All Purity'}],
  'ARTIST_NAME': 'Indian',
  'ARTIST_PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Indian'},
 {'ALBUMS': [{'ALBUM_NAME': "There's a Riot Goin' On",
    'ALBUM_PATH': "/Users/OGN/Music/iTunes/iTunes Music/Music/Sly and the Family Stone/There's a Riot

#### Get album art from song file or get from internet using sacad

In [4]:
def get_album_artwork(album_name, artist_name, song_file):

    artwork_file_name = "{} - {}.jpg".format(artist_name, album_name)
    rel_artwork_path = "/images/album art/" + artwork_file_name
    abs_artwork_path = "/Users/OGN/1Projects/norecords/app/assets/images/album_images/" + artwork_file_name
      
    try:
        artwork = song_file.tags["APIC:"].data
        with open(abs_artwork_path, 'wb') as img:
            img.write(artwork)
            rel_artwork_path = "/images/album art/{} - {}.jpg" \
                .format(artist_name, album_name)
    except Exception as e:
        try:
            result = subprocess.check_output('sacad "{}" "{}" 600 "{}"' \
                                        .format(artist_name, album_name, abs_artwork_path),
                                        shell=True, stderr=subprocess.STDOUT)
        except Exception, e:
            print repr(e)
        else:
            if "Downloading" not in result:
                rel_artwork_path = 'N/A' 
                print(artist_name, album_name)
    
    return rel_artwork_path

#### Create the music dictionary

In [5]:
def create_album_dict(lib):
    albums_dict = dict()
    for artist in lib:
        artist_name = artist['ARTIST_NAME']
        for album in artist['ALBUMS']:
            album_name = album['ALBUM_NAME']
            songs = [song for song in os.listdir(album['ALBUM_PATH'])
                        if not song.startswith('.')]
            if not songs:
                continue

            try:
                song_file = File('{}/{}'.format(album['ALBUM_PATH'], songs[0]))
            except Exception as e:
                pass
            
            artwork_path = get_album_artwork(album_name, artist_name, song_file)
            
            try:
                genre = str(song_file.tags["TCON"])
                genre = genre.replace('Prog.', 'Progressive').replace('Prog ', 'Progressive ')
                genre = genre.split(',')
            except:
                genre = []

            try:
                date = str(song_file.tags["TDRC"]).split('-')[0]
            except:
                date = ''
            
            
            songs = [song.replace('.mp3', '') for song in songs]
            songs = [song.replace('.m4a', '') for song in songs]
            
            albums_dict[album_name] = {
                "NAME": album_name, "PATH": album['ALBUM_PATH'],
                "ARTIST": artist_name, "GENRE": genre, "DATE": date, 
                "SONGS": songs, "IMAGE_PATH": artwork_path
        }     
    return albums_dict

In [6]:
albums_dict = create_album_dict(music_lib)

('Wu-Tang Clan', 'Limak n Lime')
('Limak n Lime', 'Limak n Lime')
('Oum Kolthoum', 'Yalli Kan Yeshgeek Anni')
('The Daniel Pemberton TV Orchestra', 'Little Big Planet_ Little Big Music Musical (Soundtrack from Video Game)')
('Ro\xcc\x88yksopp', 'Melody A.M_')
('Geto Boys', 'Comp CD 4')
('Beastie Boys', 'Limak n Lime')
('Al-Safi', 'The Best of Al-Sahi')
('Jan Dlabal', "Jan Dlabal's Album")
('Knxwledge_', 'klemintine_taype')
('Knxwledge_', 'karma.loops.prt.1')
('Knxwledge_', "[DB]FiXXX's")
('Compilations', 'Jazz re_freshed Live - Prelude')
('Limako', 'Kids (1995) Samples')
('Limako', 'Logxecokeh')
('Limako', 'The Camsound')
('Marc-Andre\xcc\x81 Dalbavie', 'Dalbavie - Color, Violin Concerto, Ciaccona')


#### Prune unusable entries

In [7]:
for album in albums_dict.items():
    if album[1]['IMAGE_PATH'] == 'N/A':
        del albums_dict[album[0]]            

### Show the created albums dict

In [8]:
df = pd.DataFrame(albums_dict)
df = df.T
df

,ARTIST,DATE,GENRE,IMAGE_PATH,NAME,PATH,SONGS
#3,Finalzinho Chegando,2012,[Deep House],/images/album art/Finalzinho Chegando - #3.jpg,#3,/Users/OGN/Music/iTunes/iTunes Music/Music/Fin...,"[02 Pesos Teoréticos, 01 Estruturas Potencial..."
#4,Finalzinho Chegando,2012,[Deep House],/images/album art/Finalzinho Chegando - #4.jpg,#4,/Users/OGN/Music/iTunes/iTunes Music/Music/Fin...,"[02 Eu e Claudinha, Pra Sempre (Em Abell 39), ..."
'Nard,Bernard Wright,1981,[Funk],/images/album art/Bernard Wright - 'Nard.jpg,'Nard,/Users/OGN/Music/iTunes/iTunes Music/Music/Ber...,"[03 Music is the Key, 05 Just Chillin' Out, 01..."
'Round About Midnight,Miles Davis,1957,[Jazz],/images/album art/Miles Davis - 'Round About M...,'Round About Midnight,/Users/OGN/Music/iTunes/iTunes Music/Music/Mil...,"[04 Bye Bye Blackbird, 03 All of You, 05 Tadd'..."
(Pronounced 'Lĕh-'nérd 'Skin-'nérd),Lynyrd Skynyrd,1973,[Country Rock],/images/album art/Lynyrd Skynyrd - (Pronounced...,(Pronounced 'Lĕh-'nérd 'Skin-'nérd),/Users/OGN/Music/iTunes/iTunes Music/Music/Lyn...,"[03 Gimme Three Steps, 06 Mississippi Kid, 08 ..."
0181,Four Tet,2013,[Electronic],/images/album art/Four Tet - 0181.jpg,0181,/Users/OGN/Music/iTunes/iTunes Music/Music/Fou...,"[08 Track H, 06 Track F, 03 Track C, 14 Track ..."
08-09,Repeat Pattern,2011,"[Hip Hop, Instrumental]",/images/album art/Repeat Pattern - 08-09.jpg,08-09,/Users/OGN/Music/iTunes/iTunes Music/Music/Rep...,"[10 ATH, 13 BBB, 01 RLFS3, 15 MV, 04 STAB, 08 ..."
10 + 2_ 12 American Text Sound Pieces,Compilations,1975,[Experimental],/images/album art/Compilations - 10 + 2_ 12 Am...,10 + 2_ 12 American Text Sound Pieces,/Users/OGN/Music/iTunes/iTunes Music/Music/Com...,"[01 Just, 13 Crickets, 05 Give It to Me, Baby,..."
13.0.0.0.0,TTNG,2013,[Math Rock],/images/album art/TTNG - 13.0.0.0.0.jpg,13.0.0.0.0,/Users/OGN/Music/iTunes/iTunes Music/Music/TTN...,"[08 Triptych, 07 Nice Riff, Clichard, 02 Havoc..."
154,Wire,1979,[Post-Punk],/images/album art/Wire - 154.jpg,154,/Users/OGN/Music/iTunes/iTunes Music/Music/Wir...,"[02 Two People in a Room, 13 40 Versions, 09 B..."


#### Save to pickle

In [9]:
pickle.dump(albums_dict, open("albumsDict.p" , "wb"))

#### Load with pickle

In [17]:
albums_dict_list = pickle.load(open("albumsDictList.p", "rb"))

### Collect youtube links for all the songs in the music library
#### By running collectYoutubeLinks.py from the commandline

In [45]:
#pickle.dump(albums_dict_list, open("albumsDictList.p", "wb"))

In [94]:
albums_dict_list = pickle.load(open("./albumsDictList.p", "rb"))
albums_dict_list[5]

('0181',
 {'ARTIST': 'Four Tet',
  'DATE': '2013',
  'GENRE': ['Electronic'],
  'IMAGE_PATH': '/images/album art/Four Tet - 0181.jpg',
  'NAME': '0181',
  'PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Four Tet/0181',
  'SONGS': ['08 Track H',
   '06 Track F',
   '03 Track C',
   '14 Track N',
   '16 Track P',
   '11 Track K',
   'Four Tet - 0181',
   '05 Track E',
   '12 Track L',
   '17 Track Q',
   '10 Track J',
   '15 Track O',
   '09 Track I',
   '02 Track B',
   '07 Track G',
   '13 Track M',
   '04 Track D',
   '01 Track A'],
  'SONGS_DICT': [{'TITLE': '08 Track H',
    'YT_LINK': ['https://youtube.com/watch?v=o2UPftTP1Ig?autoplay=1']},
   {'TITLE': '06 Track F',
    'YT_LINK': ['https://youtube.com/watch?v=zbhafEipo6c?autoplay=1']},
   {'TITLE': '03 Track C',
    'YT_LINK': ['https://youtube.com/watch?v=lWInZ4N6C2g?autoplay=1']},
   {'TITLE': '14 Track N',
    'YT_LINK': ['https://youtube.com/watch?v=s_Bn6FlvflM?autoplay=1']},
   {'TITLE': '16 Track P',
    'YT_LINK': ['ht

In [27]:
for album in albums_dict_list[0:10]:
    for song in album[1]['SONGS_DICT']:
        print album[1]['ARTIST'], ",", album[1]['NAME'], ",", song['TITLE'], song['YT_LINK']
    print album

Finalzinho Chegando , #3 , 02 Pesos Teoréticos []
Finalzinho Chegando , #3 , 01 Estruturas Potencializantes, Megalópolis de Desistências []
('#3', {'IMAGE_PATH': '/images/album art/Finalzinho Chegando - #3.jpg', 'ARTIST': 'Finalzinho Chegando', 'SONGS_DICT': [{'YT_LINK': [], 'TITLE': '02 Pesos Teore\xcc\x81ticos'}, {'YT_LINK': [], 'TITLE': '01 Estruturas Potencializantes, Megalo\xcc\x81polis de Desiste\xcc\x82ncias'}], 'GENRE': ['Deep House'], 'DATE': '2012', 'PATH': '/Users/OGN/Music/iTunes/iTunes Music/Music/Finalzinho Chegando/#3', 'SONGS': ['02 Pesos Teore\xcc\x81ticos', '01 Estruturas Potencializantes, Megalo\xcc\x81polis de Desiste\xcc\x82ncias'], 'NAME': '#3'})
Finalzinho Chegando , #4 , 02 Eu e Claudinha, Pra Sempre (Em Abell 39) []
Finalzinho Chegando , #4 , 01 Volta e Sonda ['https://youtube.com/watch?v=pds10dlkKrk?autoplay=1']
('#4', {'IMAGE_PATH': '/images/album art/Finalzinho Chegando - #4.jpg', 'ARTIST': 'Finalzinho Chegando', 'SONGS_DICT': [{'YT_LINK': [], 'TITLE': '0

##### Prune albums with too few song links

In [101]:
def create_final_albums_list(albums_dict_list):   
    for album in albums_dict_list:
        if not album[1]['DATE']:
            album[1]['DATE'] = 0
        
        yt_count = 0
        album[1]['SONGS_DICT'] = sorted(album[1]['SONGS_DICT'], key=lambda x: x['TITLE'])
        for song in album[1]['SONGS_DICT']:
            try:
                song['YT_LINK'][0] = song['YT_LINK'][0].replace('watch?v=',  'embed/')
            except:
                pass
            if song['YT_LINK']:
                yt_count += 1
        if yt_count > len(album[1]['SONGS_DICT']) // 2:
            yield album[1]
        else:
            print album[0]

In [103]:
final_albums_list = list(create_final_albums_list(albums_dict_list))
print len(albums_dict_list), len(final_albums_list)

#3
#4
A Child Called It
A Child Called _It_
Been Traïeying Not to Shhhyvr
CPCT Samples
Come Around Go Around
Cosmos Purple Coal Theater - Interludes
Cosmos, Purple, Coal Theater
Haydn_ Piano Sonatas
Haydn_ Theresienmesse, Heiligmesse
Hell ___ Mizmor
I Bet You It Was That One
Interpenetrating Dimensional Express
Jzivilitic Overtones
Krankenhaus Klänge
Life After Death
Limak n Lime
Lingua Franca
More Skin With Milk-Mouth
Mozart_ Die Zauberflöte (Jacobs)
Mozart_ Sonata K 448; Schubert_ Fantasia
Muntwelve1
Notorious (Music from and Inspired by the Original Motion Picture)
Pink Magick
Primitive Man_Xaphan - split 7_
Ready to Die
Sabîl
Selected Rooms
Symphonie pour le Jour où Brûleront les Cités
Symphonies Nos. 44, 51, 52
Those Scraps
Unknown Album
1753 1720


In [104]:
print final_albums_list[3]

{'IMAGE_PATH': '/images/album art/Four Tet - 0181.jpg', 'ARTIST': 'Four Tet', 'SONGS_DICT': [{'YT_LINK': ['https://youtube.com/embed/CIxHxdDh6Lw?autoplay=1'], 'TITLE': '01 Track A'}, {'YT_LINK': ['https://youtube.com/embed/BpXcwiWZ9Hk?autoplay=1'], 'TITLE': '02 Track B'}, {'YT_LINK': ['https://youtube.com/embed/lWInZ4N6C2g?autoplay=1'], 'TITLE': '03 Track C'}, {'YT_LINK': ['https://youtube.com/embed/JbGom_FLTCU?autoplay=1'], 'TITLE': '04 Track D'}, {'YT_LINK': ['https://youtube.com/embed/lWInZ4N6C2g?autoplay=1'], 'TITLE': '05 Track E'}, {'YT_LINK': ['https://youtube.com/embed/zbhafEipo6c?autoplay=1'], 'TITLE': '06 Track F'}, {'YT_LINK': ['https://youtube.com/embed/TUDsVxBtVIg?autoplay=1'], 'TITLE': '07 Track G'}, {'YT_LINK': ['https://youtube.com/embed/o2UPftTP1Ig?autoplay=1'], 'TITLE': '08 Track H'}, {'YT_LINK': ['https://youtube.com/embed/CIxHxdDh6Lw?autoplay=1'], 'TITLE': '09 Track I'}, {'YT_LINK': ['https://youtube.com/embed/CIxHxdDh6Lw?autoplay=1'], 'TITLE': '10 Track J'}, {'YT_LI

In [105]:
pickle.dump(final_albums_list, open("finalAlbumsList.p", "wb"))

In [45]:
final_albums_list = pickle.load(open("./finalAlbumsList.p", "rb"))

In [47]:
for album in final_albums_list:
    path = album['IMAGE_PATH'].decode('utf-8')
    path = unidecode.unidecode(path)
    album['IMAGE_PATH'] = path

In [48]:
pickle.dump(final_albums_list, open("finalAlbumsListUnaccented.p", "wb"))

In [49]:
rel = '../app/assets/images/album art/'
album_images = [rel + a for a in os.listdir(rel)]
for image in album_images:
    path = image.decode('utf-8')
    new_path = unidecode.unidecode(path)
    os.rename(image, new_path)

### Create PostgreSQL database

In [50]:
import psycopg2, pickle

In [51]:
final_albums_list = pickle.load(open('finalAlbumsListUnaccented.p', 'rb'))
final_albums_list = sorted(final_albums_list, key=lambda x: x['ARTIST'])

In [52]:
conn = psycopg2.connect(dbname='norecordsdb', user='gqe', host='localhost', password='')
cursor = conn.cursor()

In [53]:
create_table_sql = (
                'DROP TABLE IF EXISTS songs;'
                'CREATE TABLE songs ('
                'song_id serial PRIMARY KEY,'
                'song_name text NOT NULL,'
                'artist text NOT NULL,'
                'album text NOT NULL,'
                'genre text NOT NULL,'
                'year smallint NOT NULL,'
                'youtube_link text NOT NULL,'
                'artwork_path text NOT NULL);'
                )

In [54]:
cursor.execute(create_table_sql)

In [55]:
cursor.close()
conn.commit()

In [56]:
def create_insert_commands(final_albums_list, start=0):
    counter = start
    for album in final_albums_list:    
        for song in album['SONGS_DICT']:
            
            if song['YT_LINK']:
                song_yt_link = song['YT_LINK'][0]
            else:
                song_yt_link = 'N/A'

            insert_command = (
                """
                    INSERT INTO songs (song_id, song_name, artist, album, year, genre, youtube_link, artwork_path) 
                    VALUES ({}, '{}', '{}', '{}', {}, '{}', '{}', '{}');
                """ .format(
                        counter, 
                        song['TITLE'].replace("'", "''"), 
                        album['ARTIST'].replace("'", "''"), 
                        album['NAME'].replace("'","''"), 
                        album['DATE'], 
                        ",".join(album['GENRE']).replace("'","''"),
                        song_yt_link, 
                        album['IMAGE_PATH'].replace("'", "''")) )
            yield insert_command
            counter += 1

In [57]:
insert_commands = list(create_insert_commands(final_albums_list, 0))

In [58]:
print insert_commands[:32]

["\n                    INSERT INTO songs (song_id, song_name, artist, album, year, genre, youtube_link, artwork_path) \n                    VALUES (0, '01 Poison Root', '(Sandy) Alex G', 'Rocket', 2017, 'Indie Rock', 'https://youtube.com/embed/8lTEqoxbHaA?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg');\n                ", "\n                    INSERT INTO songs (song_id, song_name, artist, album, year, genre, youtube_link, artwork_path) \n                    VALUES (1, '02 Proud', '(Sandy) Alex G', 'Rocket', 2017, 'Indie Rock', 'https://youtube.com/embed/-vxOzwDEXv0?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg');\n                ", "\n                    INSERT INTO songs (song_id, song_name, artist, album, year, genre, youtube_link, artwork_path) \n                    VALUES (2, '03 County', '(Sandy) Alex G', 'Rocket', 2017, 'Indie Rock', 'https://youtube.com/embed/fG9RJ4y7HOY?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg');\n            

In [61]:
conn = psycopg2.connect(dbname='norecordsdb', user='gqe', host='localhost', password='')
cursor = conn.cursor()

for insert_command in insert_commands:
    try:
        cursor.execute(insert_command)
    except Exception, e:
        print e

In [62]:
cursor.close()
conn.commit()

#### Query created table

In [66]:
conn = psycopg2.connect(dbname='norecordsdb', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute('SELECT * FROM songs ORDER BY song_id')

rows = cursor.fetchall()
for row in rows[:600]:
    print(row)

(0, '01 Poison Root', '(Sandy) Alex G', 'Rocket', 'Indie Rock', 2017, 'https://youtube.com/embed/8lTEqoxbHaA?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg')
(1, '02 Proud', '(Sandy) Alex G', 'Rocket', 'Indie Rock', 2017, 'https://youtube.com/embed/-vxOzwDEXv0?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg')
(2, '03 County', '(Sandy) Alex G', 'Rocket', 'Indie Rock', 2017, 'https://youtube.com/embed/fG9RJ4y7HOY?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg')
(3, '04 Bobby', '(Sandy) Alex G', 'Rocket', 'Indie Rock', 2017, 'https://youtube.com/embed/nPuxLpVus-k?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg')
(4, '05 Witch', '(Sandy) Alex G', 'Rocket', 'Indie Rock', 2017, 'https://youtube.com/embed/G40rzZOHna8?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg')
(5, '06 Horse', '(Sandy) Alex G', 'Rocket', 'Indie Rock', 2017, 'https://youtube.com/embed/Ycc62mcMCMU?autoplay=1', '/images/album art/(Sandy) Alex G - Rocket.jpg')
(6,

In [64]:
conn = psycopg2.connect(dbname='norecordsdb', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute("UPDATE songs SET genre='Arabic Traditional' WHERE genre='Arabic';")

cursor.close()
conn.commit()

In [64]:
conn = psycopg2.connect(dbname='norecordsdb', user='gqe', host='localhost', password='')
cursor = conn.cursor()
cursor.execute("DELETE FROM songs WHERE ='Arabic Traditional' WHERE genre='Arabic';")

cursor.close()
conn.commit()